# Sachs Protein Signaling Network

Causal discovery on the Sachs dataset.


In [ ]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)


In [ ]:
from src.data_loader import download_sachs
from src.discovery import compare_discovery_methods, get_sachs_ground_truth
from src.viz import plot_discovery_metrics

data_path = download_sachs()
if data_path:
    data = pd.read_csv(data_path)
else:
    from causallearn.dataset import load_sachs
    data = pd.DataFrame(load_sachs())

print(f"Loaded {len(data)} rows, {len(data.columns)} columns")


In [ ]:
true_adj, node_names = get_sachs_ground_truth()

if data.shape[1] == len(node_names):
    data.columns = node_names[:data.shape[1]]

results = compare_discovery_methods(
    data,
    true_adjacency=true_adj,
    methods=["pc", "fci", "notears"],
    output_dir=Path("../results/sachs"),
    node_names=list(data.columns)[:len(node_names)]
)

print("\nDiscovery Results:")
print(results.to_string())

# Plot metrics
if not results.empty:
    metrics_dict = {
        row["method"]: {
            "precision": row["precision"],
            "recall": row["recall"]
        }
        for _, row in results.iterrows()
    }
    plot_discovery_metrics(
        metrics_dict,
        output_path=Path("../results/sachs/sample_plots/discovery_metrics.png")
    )
